<a href="https://colab.research.google.com/github/pyth1245/Beauty-App-Millaud/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import requests
from datetime import datetime, timezone

import streamlit as st

# === CONFIG (from info.txt) ===
API_DEFAULT = (
    "https://script.google.com/macros/s/"
    "AKfycbyNZNOE1DYNbd4GbGTISJsGrnJ4PYCuip0yjSw3Lr8KkD6-kadKI9mfpKNfiAHEWb0Osw"
    "/exec"
)

# Same deadlines / openings as the console clients
# Commit deadline (Paris 30 Nov 2025 23:59:59) == 30 Nov 2025 22:59:59 UTC
COMMIT_DEADLINE_UTC = datetime(2025, 11, 30, 22, 59, 59, tzinfo=timezone.utc)

# Reveal open (Paris 22 Oct 2025 00:00:00) == 21 Oct 2025 22:00:00 UTC
REVEAL_OPEN_UTC = datetime(2025, 10, 21, 22, 0, 0, tzinfo=timezone.utc)


# === HELPERS ===
def sha256(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()


def now_utc() -> datetime:
    return datetime.now(timezone.utc)


# === STREAMLIT UI ===
st.set_page_config(
    page_title="NEOMA Beauty Contest – Commit & Reveal",
    page_icon="🎯",
    layout="centered",
)

st.title("🎯 Keynesian Beauty Contest – Commit & Reveal")
st.write(
    """
This app lets you **commit** and **reveal** your guess for the class beauty contest,
using the same API and rules as the Python console scripts.
"""
)

# Let the student override the API URL if needed
with st.expander("API settings (optional)"):
    api_url = st.text_input(
        "Google Apps Script API URL (leave as default unless told otherwise)",
        value=API_DEFAULT,
    )
else:
    api_url = api_url if "api_url" in locals() else API_DEFAULT

phase = st.radio(
    "Choose phase",
    ["Commit (at home, before deadline)", "Reveal (in class / after opening)"],
    horizontal=False,
)

st.markdown("---")


# === COMMIT PHASE UI ===
if phase.startswith("Commit"):
    st.subheader("🔐 Commit Phase")

    current = now_utc()
    st.info(
        f"Commit deadline (Paris time): **30 Nov 2025, 23:59:59**  \n"
        f"Commit deadline (UTC): **30 Nov 2025, 22:59:59**"
    )

    if current > COMMIT_DEADLINE_UTC:
        st.error("⛔ Commit window is CLOSED. You can no longer submit commits.")
    else:
        st.success("✅ Commit window is currently **OPEN**.")

        uni_id = st.text_input("Your NEOMA / University ID")
        number = st.number_input(
            "Choose your number (0–100)", min_value=0, max_value=100, step=1
        )
        nonce = st.text_input(
            "Choose a secret nonce (keep it safe – you'll need it to reveal)",
            type="password",
        )

        if st.button("Submit Commit"):
            # Basic validation
            if not uni_id.strip():
                st.error("Please enter your University ID.")
            elif not str(nonce).strip():
                st.error("Nonce cannot be empty.")
            else:
                # Compute hash & preimage like in commit_client.py
                preimage = f"{uni_id.strip()}|{int(number)}|{nonce.strip()}"
                commit_hash = sha256(preimage)

                st.write("### ✅ Your Commit Details")
                st.write("**Preimage (write this down carefully!):**")
                st.code(preimage, language="text")
                st.write("**Commit hash:**")
                st.code(commit_hash, language="text")

                payload = {
                    "kind": "commit",
                    "uni_id": uni_id.strip(),
                    "commit": commit_hash,
                }

                try:
                    resp = requests.post(api_url, json=payload, timeout=15)
                    st.write("### 🌐 Server Response")
                    st.write(f"Status code: `{resp.status_code}`")
                    st.code(resp.text, language="text")
                except Exception as e:
                    st.error(f"Network error while sending commit: {e}")


# === REVEAL PHASE UI ===
else:
    st.subheader("📤 Reveal Phase")

    current = now_utc()
    st.info(
        f"Reveal opens (Paris time): **22 Oct 2025, 00:00:00**  \n"
        f"Reveal opens (UTC): **21 Oct 2025, 22:00:00**"
    )

    if current < REVEAL_OPEN_UTC:
        st.error("⛔ Reveal window is NOT open yet. Please come back later.")
    else:
        st.success("✅ Reveal window is currently **OPEN**.")

        st.write("Re-enter **exactly** what you used for your commit:")

        uni_id = st.text_input("Your NEOMA / University ID", key="reveal_uni_id")
        number = st.number_input(
            "Your number (0–100)", min_value=0, max_value=100, step=1, key="reveal_num"
        )
        nonce = st.text_input(
            "Your secret nonce", type="password", key="reveal_nonce"
        )

        if st.button("Submit Reveal"):
            if not uni_id.strip():
                st.error("Please enter your University ID.")
            elif not str(nonce).strip():
                st.error("Nonce cannot be empty.")
            else:
                payload = {
                    "kind": "reveal",
                    "uni_id": uni_id.strip(),
                    "number": int(number),
                    "nonce": nonce.strip(),
                }

                try:
                    resp = requests.post(api_url, json=payload, timeout=15)
                    st.write("### 🌐 Server Response")
                    st.write(f"Status code: `{resp.status_code}`")
                    st.code(resp.text, language="text")
                except Exception as e:
                    st.error(f"Network error while sending reveal: {e}")


# === EXTRA: Links to shared ledger ===
st.markdown("---")
st.caption(
    "You can independently verify the shared ledger (blockchain-style) by viewing "
    "[commits](%(base)s?table=commits) and "
    "[reveals](%(base)s?table=reveals) in your browser."
    % {"base": API_DEFAULT}
)
